In [4]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
"You will now see all output from Python cells."
"Run twice."    

'You will now see all output from Python cells.'

'Run twice.'

In [5]:
from sys import version_info
print(version_info[:3])

(3, 9, 13)


# Python version: (X, Y, Z)

In [183]:
import math
import numpy
from numpy import random
import numpy as np

In [184]:
class Asset:
    """An Asset of an Option.
    
    Parameters
    ----------
    path_length : int
        the number of time steps the path is divided
    final_time : float
        the time at which the predicted value of the Asset is calculated
    interest_rate : float
        the inetrest rate of the Asset, less than 1 
    volatility : float
        the volatility of the Asset, less than 1
        
    Returns
    -------
    float
        asset value at final time
    
    Attributes
    ----------
    name: str
        the name of the Asset
    current_price: float
        the current price of the Asset
    dividend_yield: float, default = 0
        the dividend yield of the Asset
    """
    
    def __init__(self, name: str, current_price: float, dividend_yield: float = 0):
        """Initialises Asset with names, current price, and dividend yield."""
        self.name = name
        self.current_price = current_price
        self.dividend_yield = dividend_yield
    
    def simulate_path(self, path_length: int, final_time: float, *, interest_rate: float, volatility: float):
        """Returns the Asset value at the final time simulating asset path."""
        i = 0
        asset_set = [self.current_price]
        while i < path_length:
            asset_set.append(asset_set[i]*np.exp((interest_rate-((volatility**2)/2))*(final_time/path_length)+volatility
                                                           *random.normal(0, math.sqrt(final_time/path_length))))
            i += 1
        return asset_set[-1]

In [185]:
class Option(Asset):
    """An Option in general.
    
    Attributes
    ----------
    name: str
        the name of the Option
    underlying: Asset
        the underlying Asset of the Option
    exercise_price: float
        the exercise price of the Option
    option_type: str
        the option type of the Option
    maturity_time: float
        the maturity time of the Option
    
    Raises
    ------
    NameError
        If the value of the option type is not valid. It can be either "call" or "put".
    """
    
    def __init__(self, name: str, underlying: Asset, *, exercise_price: float, option_type: str, maturity_time: float):
        """Initialises Option with names, underlying, exercise price, option type and maturity time."""
        self.name = name
        self.underlying = underlying
        self.exercise_price = exercise_price
        self.option_type = self._is_valid_option_type(option_type) 
        self.maturity_time = maturity_time
        
    def _is_valid_option_type(self, option_type):
        """Checks if the option type is correct to work with."""
        if option_type == 'call' or option_type == 'put':
            return option_type
        else:
            raise NameError('Option type should be either "call" or "put"')

In [186]:
class PathIndependentOption(Option):
    """A Path independent Option in general.
    
    Parameters
    ----------
    exercise_price : float
        the exercise price of the Option
    current_price : float
        the value of the underlying asset
    option_type : str
        the option type that could be either 'call' or 'put'

    Returns
    -------
    float
        The payoff value of the Path independent Option.
    """
    
    def __init__(self, *args, **kwargs):
        """Initialises PathIndependentOption taking all instance attributes from Option class."""
        super().__init__(*args, **kwargs)
    
    def payoff(self):
        """Returns payoff of the option relying on the option type."""
        if self.option_type == 'call':
            return max(self.underlying.current_price - self.exercise_price, 0)
        else:
            return max(self.exercise_price - self.underlying.current_price, 0)
        
    @staticmethod
    def _payoff(exercise_price, current_price, option_type):
        """Returns payoff of the option relying on the option type."""
        if option_type == 'call':           
            return max(current_price - exercise_price, 0)       
        if option_type == 'put':
            return max(exercise_price - current_price, 0)

In [187]:
class BinomialValuedOption(PathIndependentOption):
    """Binomial method of valuing Options.
    
    Parameters
    ----------
    n: int
        the maximum level in the tree 
    u: float, default = None
        the factor that the underlying price is multiplied by when the price moves up 
    d: float, default = None
        the factor that the underlying price is multiplied by when the price moves down 
    p: float, default = None
        the probability that the underlying price moves up
    interest_rate: float
        the annualised, continuously-compounded interest rate
    volatility: float
        the annualised volatility
    method: str, default = None
        the method of constructing the binomial tree

    Returns
    -------
    float
        The Option value at the current time.
        
    Raises
    ------
    ValueError
        If method is provided, none of u, p, d should be provided, and if method is not provided, 
        then all of u, p, d must be provided.
    NameError
        If the value of the method is not valid. It can be either "symmetrical" or "equal probability".
    
    Notes
    -----
    Use pseudoprivate method _binomial_node_value from subclasses to calculate the value of different options.
    """
    
    def __init__(self, *args, **kwargs):
        """Initialises BinomialValuedOption taking all instance attributes from PathIndependentOption class."""
        super().__init__(*args, **kwargs)
        
    def binomial_value(self, n: int, u: float = None, d: float = None, p: float = None,
                       *, interest_rate: float, volatility: float, method: str = None):
        """Returns the value of the option using Binomial method."""
        if method != None and (u != None or d != None or p != None):
            raise ValueError('When parameter "method" is provided, non of the values "u", "d", "p" should be given')
        if method == None and (u == None or d == None or p == None):
            raise ValueError('When parameter "method" is not provided, all of the values "u", "d", "p" should be given')
        if method != None and method !='symmetrical' and method != 'equal probability':
            raise NameError('Parameter "method" can be either "symmetrical" or "equal probability"')
        dt = self.maturity_time/n
        if method != None:
            if method == 'symmetrical':
                A = (math.exp(-interest_rate*dt) + math.exp((interest_rate + volatility**2)*dt))/2
                u = A + math.sqrt(A**2 - 1)
                d = A - math.sqrt(A**2 - 1)
                p = (math.exp(interest_rate*dt) - d)/(u - d)
            if method == 'equal probability':
                p = 1/2
                u = math.exp(interest_rate*dt)*(1 + math.sqrt(math.exp((volatility**2)*dt)-1))
                d = math.exp(interest_rate*dt)*(1 - math.sqrt(math.exp((volatility**2)*dt)-1))
            if u < 1 or d < 0 or d > 1 or p < 0 or p > 1:
                raise ValueError('Choose different "n" to make dt small enought')
        i = 0
        final_asset_set = []
        while i < n+1:
            final_asset_set.append((d**(n-i))*(u**i)*self.underlying.current_price)
            
            i += 1
        v_set = []
        i = 0
        while i < n+1:
            v_set.append(PathIndependentOption._payoff(self.exercise_price, final_asset_set[i], self.option_type))
            i += 1
        def calculate_option_value(n, p, *, v_set, interest_rate):
            if n == 0:
                return v_set[0]
            else:
                i = 0
                v_new_set = []
                while i < n:
                    v_new_set.append(self._binomial_node_value(v_set[i+1],v_set[i], p,
                                                               -interest_rate*dt, (d**(n-i))*(u**i)*self.underlying.current_price))
                    i += 1
                return calculate_option_value(n - 1, p, v_set = v_new_set, interest_rate = interest_rate)    
        return calculate_option_value(n, p, v_set = v_set, interest_rate = interest_rate)

In [188]:
class MonteCarloValuedOption(Option):
    """Monte Carlo method of valuing Options.
    
    Parameters
    ----------
    num_paths : int
        the number of simulated paths to construct
    path_length : int
        the length of the simulated paths
    interest_rate : float
        the annualised, continuously-compounded interest rate
    volatility : float
        the annualised volatility

    Returns
    -------
    float
        The Option value at the current time.
    
    Notes
    -----
    Use pseudoprivate method _monte_carlo_sim_value from subclasses to calculate the value of different options.
    """
    
    def __init__(self, *args, **kwargs):
        """Initialises MonteCarloValuedOption taking all instance attributes from Option class."""
        super().__init__(*args, **kwargs)
        
    def monte_carlo_value(self, num_paths, path_length, *, interest_rate, volatility):
        """Returns the value of the option using Monte Carlo method."""
        i = 0
        asset_values = []
        new_asset = Asset(self.name, self.underlying.current_price)
        while i < num_paths:
            asset_values.append(new_asset.simulate_path(path_length, self.maturity_time, 
                                    interest_rate = interest_rate, volatility = volatility))
            i += 1
        return self._monte_carlo_sim_value(num_paths, path_length, asset_values, 
                                                   interest_rate = interest_rate, volatility =volatility)

In [189]:
class EuropeanOption(MonteCarloValuedOption, BinomialValuedOption):
    """An European type of Options.
    
    Parameters
    ----------
    V_up : float
        the value of option at the next node when the price moves up 
    V_down : float
        the value of option at the next node when the price moves up 
    discount_factor : float
        the value of multiplication of interest_rate on the time step
    S : float
        the value of underlying
    num_paths : int
        the number of simulated paths to construct
    path_length : int
        the length of the simulated paths
    interest_rate : float
        the annualised, continuously-compounded interest rate
    volatility : float
        the annualised volatility
    asset_values: list
        the list of asset values at final time
    
    Notes
    -----
    The class has two pseudoprivate methods: _binomial_node_value and _monte_carlo_sim_value. The methods are used 
    to calculate the value of European Options in monte_carlo_value and binomial_value methods from
    MonteCarloValuedOption and BinomialValuedOption classes respectively.
    """
    
    def __init__(self, *args, **kwargs):
        """Initialises EuropeanOption taking all instance attributes from 
        MonteCarloValuedOption and BinomialValuedOption classes."""
        super().__init__(*args, **kwargs)
        
    def _binomial_node_value(self, V_up, V_down, p, discount_factor, S):
        """Returns the option value of the previous node in Binomial method for European options."""
        return math.exp(discount_factor)*(p*V_up+(1-p)*V_down)
    
    def _monte_carlo_sim_value(self, num_paths, path_length, asset_values, *, interest_rate, volatility):
        """Returns the value of the European option using Monte Carlo method."""
        i = 0
        payoff_values = []
        while i < num_paths:
            payoff_values.append(PathIndependentOption._payoff(self.exercise_price, asset_values[i], self.option_type))
            i += 1
        return math.exp(-interest_rate*self.maturity_time)*((1/num_paths)*sum(payoff_values))

In [190]:
class AmericanOption(BinomialValuedOption):
    """An American type of Options.
    
    Parameters
    ----------
    V_up : float
        the value of option at the next node when the price moves up 
    V_down : float
        the value of option at the next node when the price moves up 
    discount_factor : float
        the value of multiplication of interest_rate on the time step
    S : float
        the value of underlying
    
    Notes
    -----
    The pseudoprivate method is used to calculate the value of American Options in binomial_value methods
    in BinomialValuedOption class.
    """
    def __init__(self, *args, **kwargs):
        """Initialises AmericanOption taking all instance attributes from BinomialValuedOption class."""
        super().__init__(*args, **kwargs)
    
    def _binomial_node_value(self, V_up, V_down, p, discount_factor, S):
        """Returns the option value of the previous node in Binomial method for American options."""
        return max(math.exp(discount_factor)*(p*V_up+(1-p)*V_down), 
                   PathIndependentOption._payoff(self.exercise_price, S, self.option_type))

In [191]:
class AsianEuropeanOption(MonteCarloValuedOption):
    """An Asian type of European Options.
    
    Parameters
    ----------
    num_paths : int
        the number of simulated paths to construct
    path_length : int
        the length of the simulated paths
    interest_rate : float
        the annualised, continuously-compounded interest rate
    volatility : float
        the annualised volatility
    asset_values: list
        the list of asset values at final time
    
    Attributes
    ----------
    averaging_method: str
        the method of averaging assets values
    
    Raises
    ------
    NameError
        If the value of the averaging method is not valid. It can be either "arithmetic" or "geometric".
    
    Notes
    -----
    The pseudoprivate method is used to calculate the value of Asian type of European Options in monte_carlo_value method
    in MonteCarloValuedOption class.
    """
    def __init__(self, *args, averaging_method: str, **kwargs):
        """Initialises AsianEuropeanOption taking all instance attributes from MonteCarloValuedOption class."""
        super().__init__(*args, **kwargs)
        self.averaging_method = self._is_valid_averaging_method(averaging_method)
        
    def _is_valid_averaging_method(self, averaging_method):
        """Checks if the averaging method is correct to work with."""
        if averaging_method == 'arithmetic' or averaging_method == 'geometric':
            return averaging_method
        else:
            raise NameError('Averaging method should be either "arithmetic" or "geometric"')
    
    def _monte_carlo_sim_value(self, num_paths, path_length, asset_values, *, interest_rate, volatility):
        """Returns the value of the Asian option using Monte Carlo method."""
        if self.averaging_method == 'arithmetic':
            asian_exercise_price = (1/num_paths)*sum(asset_values)
        if self.averaging_method == 'geometric':
            asian_exercise_price = numpy.prod(asset_values)**(1/num_paths)
        i = 0
        payoff_values = []
        while i < num_paths:
            payoff_values.append(PathIndependentOption._payoff(asian_exercise_price, asset_values[i], self.option_type))
            i += 1
        return math.exp(-interest_rate*self.maturity_time)*((1/num_paths)*sum(payoff_values))

In [192]:
if __name__ == '__main__':
    print('Checking if simulate_path works')
    first_attempt = Asset('first', 200)
    first_attempt.simulate_path(200, 9, interest_rate = 0.01, volatility = 0.1)

    print('Checking if Option class works')
    new0= Asset('first', 200)
    new0 = Option('Call', first_attempt, exercise_price = 56, option_type = "call", maturity_time = 7)
    new0.underlying
    
    print('Checking if PathIndependentOption work correctly')
    new1 = PathIndependentOption('Call', first_attempt, exercise_price = 56, option_type = "call", maturity_time = 7)
    new1.payoff()
    new1._payoff(120, 100, 'call')
    PathIndependentOption._payoff(120, 100, 'call')
    
    print('Checking if the methods work correctly according to EuropeanOption')
    new4 = EuropeanOption('Call', first_attempt, exercise_price = 40, option_type = 'put', maturity_time = 3)
    new4.monte_carlo_value(10000, 10, interest_rate = 0.1, volatility = 0.1)
    new4.binomial_value(3, interest_rate = 0.05, volatility = 0.5, method = 'equal probability')

    print('Checking if the AmericanOption class works')
    new5 = AmericanOption('Call', first_attempt, exercise_price = 100, option_type = 'call', maturity_time = 7)
    new5.binomial_value(3, interest_rate = 0.05, volatility = 0.1, method = 'symmetrical')
    
    print('Checking if the AsianOption class works for two averaging methods arithmetic and geometric respectively')
    new6 = AsianEuropeanOption('Call', first_attempt, exercise_price = 100, 
                               option_type = 'call', maturity_time = 7, averaging_method = 'arithmetic')
    new6.monte_carlo_value(5, 10, interest_rate = 0.1, volatility = 0.1)
    new6=AsianEuropeanOption('Call', first_attempt, exercise_price = 100, 
                               option_type = 'call', maturity_time = 7, averaging_method = 'geometric')
    new6.monte_carlo_value(5, 10, interest_rate = 0.1, volatility = 0.1)

Checking if simulate_path works


201.20547545257617

Checking if Option class works


Checking if PathIndependentOption work correctly


144

...

Checking if the AmericanOption class works


129.53119102812866

Checking if the AsianOption class works for two averaging methods arithmetic and geometric respectively


11.836280498390828

12.024201254250766